# Spotify Streams Visualizer

Visualizes hyped spotify songs, so you may never miss the newest truly aweful song that everyone knows. 

First streams per day of the 200 most streamed songs from www.spotifycharts.com are crawled. Then the most streamed songs played in the last view days are plotted.  

In [ ]:
import pandas as pd
import io
import requests
import datetime
import numpy as np

def download_spotify_day(url):
    s = requests.get(url).content
    s = s.decode('utf-8')[127:] # remove first line of file - not needed
    df = pd.read_csv(io.StringIO(s))
    return df


def get_dates(num_days=10, day_delta=1):
    base = datetime.datetime.today()
    return list(reversed([base - datetime.timedelta(days=x*day_delta) for x in range(num_days+1)][1:]))

In [ ]:
# download csv files with stream data and return dataframes. Crawl responsibly.
dates = get_dates(num_days=50, day_delta=1) # remove today

dfs = []
streams = []
for i, date in enumerate(dates):
    print('download: ', i)
    url = "https://spotifycharts.com/regional/global/daily/{}/download".format(date.strftime("%Y-%m-%d"))
    df = download_spotify_day(url)
    dfs.append(df)


In [ ]:
# filter and rename
datas = []
for i,df in enumerate(dfs):
    data = df[["Track Name", "Streams"]]
    
    # rename cols
    data = data.rename(columns={"Track Name": "track", "Streams": str(i)})
    datas.append(data)
    
# create relevant songs
songs = datas[0]['track'].tolist()[:20]
songs

In [ ]:
# for all songs, get streams per day

song_streams = {} # song: stream_list
for song in songs:
    song_streams[song] = []
    
for data in datas:
    for song in songs:
        streamed = data[data['track'] == song]
        if len(streamed) == 1:
            streamed = streamed.iloc[0][1]
        else:
            streamed = 0
        song_streams[song].append(streamed)


In [ ]:
plt.figure(figsize=(10, 5))
for song in songs[:10]:
    plt.plot(dates, song_streams[song], label=song)

plt.legend(title='Songs', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Spotify - Most Streamed Songs')
plt.ylabel('Streams per Day')
plt.xticks(rotation=20)
plt.tight_layout()

plt.savefig('spotify.png', dpi=300)